In [1]:
# =============================================================================
# Section 1: Import Libraries
# =============================================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from joblib import dump
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# Set plotting style for better visuals
sns.set_style("whitegrid")
print("Libraries imported successfully.")


# =============================================================================
# Section 2: Data Loading, Cleaning, and Merging for MEPFS Data
# =============================================================================
print("\n--- Section 2: Loading, Cleaning & Merging MEPFS Data ---")

def clean_and_prepare_mepfs_data(df, file_type='quantity'):
    """A reusable function to perform initial cleaning on the MEPFS dataframes."""
    df.rename(columns={df.columns[0]: 'Project_Name'}, inplace=True)
    df.drop(columns=['MEPFS aspect'], inplace=True, errors='ignore')
    df['Project_Name'] = df['Project_Name'].str.strip().str.lower()
    df['Project_Name'] = df['Project_Name'].apply(transform_project_name)
    
    mepfs_feature_cols = [
        'Fire alarm system', 'Panelboard', 'Lighting fixtures', 'Conduits',
        'Sewer Line works', 'Plumbing fixtures', 'Wires'
    ]

    for col in mepfs_feature_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.replace('-', 'NaN', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')

    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            if df[col].isnull().any():
                median_val = df[col].median()
                df[col].fillna(median_val, inplace=True)
    
    return df

def transform_project_name(text):
    """Converts patterns like '1x5' into '1 STY 5 CLS'."""
    if not isinstance(text, str): return text
    match = re.search(r'(\d+)\s*[xX]\s*(\d+)', text)
    if match:
        n, m = match.groups()
        return text.replace(match.group(0), f"{n} STY {m} CLS")
    return text

def extract_year_budget(text):
    """Extracts year and budget from the 'Year/Budget' string."""
    if not isinstance(text, str): return None, None
    year_match = re.search(r'\b(20\d{2})\b', text)
    year = int(year_match.group(1)) if year_match else None
    budget_match = re.search(r'[:\s]([\d,]+\.?\d*)', text)
    budget = float(budget_match.group(1).replace(',', '')) if budget_match else None
    return year, budget

try:
    df_quantity = pd.read_csv('MEPFS Quantity Cost.csv')
    print("MEPFS Quantity data loaded.")
    df_unit_cost = pd.read_csv('MEPFS Unit Cost.csv')
    print("MEPFS Unit Cost data loaded.")
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure both CSV files are in the correct directory.")
    exit()

# --- Clean and Merge Dataframes ---
df_quantity_cleaned = clean_and_prepare_mepfs_data(df_quantity.copy())
df_unit_cost_cleaned = clean_and_prepare_mepfs_data(df_unit_cost.copy(), file_type='unit_cost')

df_quantity_cleaned[['Year', 'Budget']] = df_quantity_cleaned[df_quantity_cleaned.columns[1]].apply(extract_year_budget).apply(pd.Series)
df_quantity_cleaned.drop(columns=[df_quantity_cleaned.columns[1]], inplace=True)

df_merged = pd.merge(
    df_quantity_cleaned,
    df_unit_cost_cleaned,
    on='Project_Name',
    suffixes=('_qty', '_cost')
)
df_merged = df_merged.dropna(subset=['Budget'])
df_merged = df_merged[df_merged['Budget'] > 100000].copy()
print(f"Dataframes merged. Working with {len(df_merged)} common projects.")


# =============================================================================
# Section 3: Granular Feature Engineering & Visualization
# =============================================================================
print("\n--- Section 3: Engineering Granular Features & Analysis ---")

# --- Step 1: Multiply Quantity by Unit Cost for Granular Features ---
total_cost_features = []
base_mepfs_cols = [
    'Fire alarm system', 'Panelboard', 'Lighting fixtures', 'Conduits',
    'Sewer Line works', 'Plumbing fixtures', 'Wires'
]

for col in base_mepfs_cols:
    qty_col = col + '_qty'
    cost_col = col + '_cost'
    new_cost_feature = col.replace(' ', '_') + '_Total_Cost'
    
    df_merged[new_cost_feature] = df_merged[qty_col] * df_merged[cost_col]
    total_cost_features.append(new_cost_feature)

print("Created new granular features for each MEPFS component:")
print(total_cost_features)

# --- Step 2: Create Contextual and Target Features ---
df_merged['Num_Storeys'] = df_merged['Project_Name'].str.extract(r'(\d+)\s*(?:sty|x)').astype(float)
df_merged['Num_Classrooms'] = df_merged['Project_Name'].str.extract(r'(?:sty|x)\s*(\d+)\s*cl').astype(float)
df_merged['Num_Storeys'].fillna(df_merged['Num_Storeys'].median(), inplace=True)
df_merged['Num_Classrooms'].fillna(df_merged['Num_Classrooms'].median(), inplace=True)
df_merged['Budget_log'] = np.log1p(df_merged['Budget'])

# --- Step 3: Visualization and Analysis ---
print("\n--- Generating and Saving Visualizations for Analysis ---")

# Create the directory for images if it doesn't exist
output_dir = 'visualization images'
os.makedirs(output_dir, exist_ok=True)

# 3.1: Justifying Log-Transformation (Section 1.1 of Report)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.histplot(df_merged['Budget'], kde=True, ax=axes[0], bins=30)
axes[0].set_title('Distribution of Original Budget (Right-Skewed)')
axes[0].set_xlabel('Budget (PHP)')
axes[0].ticklabel_format(style='plain', axis='x')
sns.histplot(df_merged['Budget_log'], kde=True, ax=axes[1], color='green', bins=30)
axes[1].set_title('Distribution of Log-Transformed Budget (Normalized)')
axes[1].set_xlabel('Log(1 + Budget)')
plt.suptitle('Effect of Log-Transformation on Target Variable', size=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(os.path.join(output_dir, 'mepfs_log_transformation_effect.png'), bbox_inches='tight')
plt.close(fig)
print(f"Saved: {os.path.join(output_dir, 'mepfs_log_transformation_effect.png')}")

# 3.2: Correlation Matrix of Final Engineered Features
plt.figure(figsize=(14, 12))
heatmap_cols = total_cost_features + ['Num_Storeys', 'Num_Classrooms', 'Budget']
correlation_matrix = df_merged[heatmap_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt='.2f', annot_kws={"size": 10})
plt.title('Correlation Matrix of MEPFS Features vs. Budget', size=16)
plt.savefig(os.path.join(output_dir, 'mepfs_features_correlation_matrix.png'), bbox_inches='tight')
plt.close()
print(f"Saved: {os.path.join(output_dir, 'mepfs_features_correlation_matrix.png')}")

# 3.3: Justifying the Need for Standardization (Section 1.3 of Report)
final_feature_columns = total_cost_features + ['Num_Storeys', 'Num_Classrooms']
X_for_viz = df_merged[final_feature_columns].copy()
X_for_viz.fillna(X_for_viz.median(), inplace=True)

plt.figure(figsize=(10, 8))
sns.boxplot(data=X_for_viz, orient='h')
plt.title('MEPFS Feature Scales Before Standardization')
plt.xlabel('Original Feature Values (Varying Scales)')
plt.xscale('log')
plt.savefig(os.path.join(output_dir, 'mepfs_feature_scales_before_standardization.png'), bbox_inches='tight')
plt.close()
print(f"Saved: {os.path.join(output_dir, 'mepfs_feature_scales_before_standardization.png')}")

# =============================================================================
# Section 4: Data Preparation for the ANN Model
# =============================================================================
print("\n--- Section 4: Preparing Data for the ANN Model ---")

# --- Define the features (X) and the target (y) ---
X = df_merged[final_feature_columns]
y = df_merged[['Budget_log']]

print(f"\nTraining model with {X.shape[1]} input features.")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Feature and Target Scaling ---
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# --- Convert to PyTorch Tensors ---
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

print("Data has been split, scaled, and converted to tensors.")


# =============================================================================
# Section 5: Build and Train the Artificial Neural Network
# =============================================================================
print("\n--- Section 5: Building and Training the ANN ---")

class RegressionNet(nn.Module):
    def __init__(self, input_features):
        super(RegressionNet, self).__init__()
        self.layer1 = nn.Linear(input_features, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.layer2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.2)
        self.layer3 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.dropout1(x)
        x = F.relu(self.layer2(x))
        x = self.dropout2(x)
        x = F.relu(self.layer3(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# --- Model Initialization and Training Setup ---
input_size = X_train_tensor.shape[1]
model = RegressionNet(input_features=input_size)
print("Model Architecture:")
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
batch_size = 16
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# --- Training Loop ---
epochs = 200
train_losses = []
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for features, targets in train_loader:
        outputs = model(features)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.6f}')

print("Neural Network training complete.")

# --- Plot and Save Training Loss ---
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.title('MEPFS Model Training Loss Over Epochs', size=16)
plt.xlabel('Epoch', size=12)
plt.ylabel('Mean Squared Error Loss', size=12)
plt.legend()
plt.savefig(os.path.join(output_dir, 'mepfs_model_training_loss_curve.png'), bbox_inches='tight')
plt.close()
print(f"Saved: {os.path.join(output_dir, 'mepfs_model_training_loss_curve.png')}")


# =============================================================================
# Section 6: Model Evaluation
# =============================================================================
print("\n--- Section 6: Evaluating the ANN Model ---")

model.eval()
with torch.no_grad():
    scaled_predictions = model(X_test_tensor).numpy()

# --- Inverse Transform Predictions to Original Scale ---
log_predictions = scaler_y.inverse_transform(scaled_predictions)
final_predictions = np.expm1(log_predictions).flatten()
y_test_actual = np.expm1(y_test.values).flatten()

# --- Calculate and Display Performance Metrics ---
r2_ann = r2_score(y_test_actual, final_predictions)
mae_ann = mean_absolute_error(y_test_actual, final_predictions)
rmse_ann = np.sqrt(mean_squared_error(y_test_actual, final_predictions))

print("\n--- Final Model Performance ---")
print(f"R-squared (R²): {r2_ann:.4f}")
print(f"Mean Absolute Error (MAE): ₱{mae_ann:,.2f}")
print(f"Root Mean Squared Error (RMSE): ₱{rmse_ann:,.2f}")

# --- Visualization: Actual vs. Predicted Budget ---
plt.figure(figsize=(10, 7))
plt.scatter(y_test_actual, final_predictions, alpha=0.7, edgecolors='k', c='royalblue', label='Predictions')
plt.plot([y_test_actual.min(), y_test_actual.max()], [y_test_actual.min(), y_test_actual.max()],
         'r--', lw=2, label='Perfect Fit')
plt.title('Actual vs. Predicted Project Budget (MEPFS Model)', size=16)
plt.xlabel('Actual Budget (PHP)', size=12)
plt.ylabel('Predicted Budget (PHP)', size=12)
plt.ticklabel_format(style='plain', axis='both')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'mepfs_actual_vs_predicted_budget.png'), bbox_inches='tight')
plt.close()
print(f"Saved: {os.path.join(output_dir, 'mepfs_actual_vs_predicted_budget.png')}")

# --- Visualization: Residuals Plot ---
residuals = y_test_actual - final_predictions
plt.figure(figsize=(10, 6))
sns.scatterplot(x=final_predictions, y=residuals, alpha=0.7, edgecolors='k', c='green')
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals Plot (MEPFS Model)')
plt.xlabel('Predicted Budget (PHP)')
plt.ylabel('Residuals (Actual - Predicted)')
plt.ticklabel_format(style='plain', axis='both')
plt.savefig(os.path.join(output_dir, 'mepfs_residuals_plot.png'), bbox_inches='tight')
plt.close()
print(f"Saved: {os.path.join(output_dir, 'mepfs_residuals_plot.png')}")

# =============================================================================
# Section 7: Save Final Assets
# =============================================================================
print("\n--- Section 7: Saving Final Model and Scalers ---")

# Save the trained model state
torch.save(model.state_dict(), 'ann_mepfs_model.pth')
print("ANN model saved as 'ann_mepfs_model.pth'")

# Save the scalers to be used for future predictions
dump(scaler_X, 'scaler_X_mepfs.joblib')
dump(scaler_y, 'scaler_y_mepfs.joblib')
print("Feature and target scalers saved.")

print("\nProcess finished successfully.")

Libraries imported successfully.

--- Section 2: Loading, Cleaning & Merging MEPFS Data ---
MEPFS Quantity data loaded.
MEPFS Unit Cost data loaded.
Dataframes merged. Working with 139 common projects.

--- Section 3: Engineering Granular Features & Analysis ---
Created new granular features for each MEPFS component:
['Fire_alarm_system_Total_Cost', 'Panelboard_Total_Cost', 'Lighting_fixtures_Total_Cost', 'Conduits_Total_Cost', 'Sewer_Line_works_Total_Cost', 'Plumbing_fixtures_Total_Cost', 'Wires_Total_Cost']

--- Generating and Saving Visualizations for Analysis ---


C:\Users\User\AppData\Local\Temp\ipykernel_60972\929080645.py:50: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_60972\929080645.py:50: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

Saved: visualization images\mepfs_log_transformation_effect.png
Saved: visualization images\mepfs_features_correlation_matrix.png
Saved: visualization images\mepfs_feature_scales_before_standardization.png

--- Section 4: Preparing Data for the ANN Model ---

Training model with 9 input features.
Data has been split, scaled, and converted to tensors.

--- Section 5: Building and Training the ANN ---
Model Architecture:
RegressionNet(
  (layer1): Linear(in_features=9, out_features=128, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (layer3): Linear(in_features=64, out_features=32, bias=True)
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
)
Epoch [20/200], Loss: 0.018895
Epoch [40/200], Loss: 0.013332
Epoch [60/200], Loss: 0.010217
Epoch [80/200], Loss: 0.007505
Epoch [100/200], Loss: 0.007882
Epoch [120/200], Loss: 0.006737
Epoch [140/200], Loss: 0.0